# INSTALLATION PYSPARK



In [2]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [824 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,265 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,077 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:13 htt

In [3]:
# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

sample_data  spark-3.1.1-bin-hadoop3.2	spark-3.1.1-bin-hadoop3.2.tgz


In [4]:
# Initialize findspark
import findspark
findspark.init()

# Create a PySpark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

# Spark SQL

## Exercice 1

- Data Diamonds : /databricks-datasets/Rdatasets/data-001/csv/ggplot2/diamonds.csv Après avoir loadé les diamants et dans l'API Spark SQL, on veut :

- Filtrer en excluant la couleur E
- Calculer le prix minimum, le prix maximum, le prix moyen en arrondissant à 2 après la virgule pour l'ensemble des diamants
- Calculer le prix minimum, le prix maximum, le prix moyen en arrondissant à 2 après la virgule par couleur

In [9]:
from pyspark import SparkFiles

path = "https://raw.githubusercontent.com/NastasiaSaby/data-sources/master/diamonds.csv"

spark.sparkContext.addFile(path)

df = spark.read.option("header", True).csv(SparkFiles.get("diamonds.csv"))

df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- carat: string (nullable = true)
 |-- cut: string (nullable = true)
 |-- color: string (nullable = true)
 |-- clarity: string (nullable = true)
 |-- depth: string (nullable = true)
 |-- table: string (nullable = true)
 |-- price: string (nullable = true)
 |-- x: string (nullable = true)
 |-- y: string (nullable = true)
 |-- z: string (nullable = true)
 |-- site: string (nullable = true)
 |-- brand: string (nullable = true)



In [11]:
df.show(5)

+---+-----+-------+-----+-------+-----+-----+-----+----+----+----+----------------+--------------+
|_c0|carat|    cut|color|clarity|depth|table|price|   x|   y|   z|            site|         brand|
+---+-----+-------+-----+-------+-----+-----+-----+----+----+----+----------------+--------------+
|  1| 0.23|  Ideal|    E|    SI2| 61.5|   55|  326|3.95|3.98|2.43|    diamonds.com|diamondForEver|
|  2| 0.21|Premium|    E|    SI1| 59.8|   61|  326|3.89|3.84|2.31|coolDiamonds.com| diamondChicos|
|  3| 0.23|   Good|    E|    VS1| 56.9|   65|  327|4.05|4.07|2.31|      diadia.com|diamondForEver|
|  4| 0.29|Premium|    I|    VS2| 62.4|   58|  334| 4.2|4.23|2.63|         dya.com| diamondChicos|
|  5| 0.31|   Good|    J|    SI2| 63.3|   58|  335|4.34|4.35|2.75|    diamonds.com| diamondChicos|
+---+-----+-------+-----+-------+-----+-----+-----+----+----+----+----------------+--------------+
only showing top 5 rows



In [17]:
from pyspark.sql.functions import col

#1
df_filter = df.where(col("color") != "E")

In [51]:
from pyspark.sql.functions import min, max, mean, round

price_mean = df.select(round(mean(col("price")), 2).alias("moyen"))

price_mean.show()

+------+
| moyen|
+------+
|3932.8|
+------+



In [59]:
df.groupBy("color").mean(col("price").cast('double')).show()



TypeError: Column is not iterable